In [1]:
from dataclasses import dataclass, field
from typing import List, Dict, Tuple
from enum import Enum
from copy import deepcopy

import numpy as np

import blackjack

In [2]:
@dataclass
class State:
    hand_total: int
    delear_partial_total: int
    
    def __hash__(self) -> int:
        return hash((self.hand_total, self.delear_partial_total))

In [3]:
class Action(Enum):
    HIT = 0
    STAND = 1

In [4]:
@dataclass
class QTable:
    table: Dict[State, List[float]] = field(default_factory=dict)
    
    def __post_init__(self) -> None:
        possible_actions = [Action.HIT, Action.STAND]
        possible_dealer_visible_totals = list(range(1, 12)) # Between 1 and 11
        possible_hand_totals = list(range(2, 23)) # If hand_total > 21, then it is 22 in the q table, no matter how bigger than 21 it is (not that it will do anything since the game will be automatically ended)
        
        for total in possible_hand_totals:
            for dealer_total in possible_dealer_visible_totals:
                state = State(total, dealer_total)
                self.table[state] = [0.0] * len(possible_actions)
    
    
    def __getitem__(self, state: State) -> List[float]:
        if state.hand_total > 21:
            state.hand_total = 22
        return self.table[state]
    
    def __setitem__(self, state: State, value: List[float]):
        if state.hand_total > 21:
            state.hand_total = 22
        self.table[state] = value
                
    def policy(self, state: State, epsilon: float = None) -> Action:
        if epsilon:
            if np.random.random() < epsilon:
                return Action(np.random.choice([0, 1]))
        return Action(np.argmax(self.table[state]))

In [5]:
class Agent:
    def __init__(self, learning_rate: float = 0.1, discount_factor: float = 0.99, epsilon_greedy: float = 0.1, epochs: int = 1000) -> None:
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon_greedy = epsilon_greedy
        self.epochs = epochs
        self.q_table = QTable()
        self.table = blackjack.Table()
        self.states: List[State] = [State(-1, -1)]
        self.episodes: List[List[Dict[str, State | Action | float]]] = []
    
    
    def reward_win_loss(self, player_total: int, dealer_total: int) -> float:
        if player_total > 21:
            return -1
        elif player_total > dealer_total:
            return 1
        elif player_total == dealer_total:
            return 0
        else:
            return -1
        
    def reward_proximity(self, player_total: int, dealer_total: int) -> float:
        player_total = player_total if player_total < 22 else 0
        return 1 / (21 - player_total + 1e-1)
                      
    
    def play_hand(self, player: blackjack.Player, current_episode: List[Dict[str, State | Action | float]] = []) -> None:
        if not player.playing:
            for i, episode in enumerate(current_episode):
                state = episode["state"]
                reward = self.reward_win_loss(state.hand_total, self.table.dealer.total)
                current_episode[i]["reward"] = reward
            return current_episode
        
        action = self.q_table.policy(State(player.hand.possible_totals[0], self.table.dealer.partial_total), self.epsilon_greedy)
        if action == Action.HIT:
            
            current_episode.append({
                "state": State(player.hand.possible_totals[0], self.table.dealer.partial_total),
                "action": action,
                "reward": 0
            })
            
            player.hit(self.table.deck)
            
        else:   
            current_episode.append({
                "state": State(player.hand.possible_totals[0], self.table.dealer.partial_total),
                "action": action,
                "reward": 0
            })
            player.stand()
        
        return self.play_hand(player, current_episode)
    
    def play_hand_possible(self, players: blackjack.RecursivePlayer = [], current_episode: List[Dict[str, State | Action | float]] = [], model_player: blackjack.Player = None) -> None:
        if len(players) == 0:
            players = [blackjack.RecursivePlayer(chosen_total, model_player.hand) for chosen_total in model_player.hand.possible_totals]
            
        print(players)
        next_card = self.table.deck.draw()
        for i, player in enumerate(players):
            if not player.playing:
                for j, episode in enumerate(current_episode):
                    state = episode["state"]
                    reward = self.reward_win_loss(state.hand_total, self.table.dealer.total)
                    current_episode[j]["reward"] = reward
                return current_episode
            
            action = self.q_table.policy(State(player.hand.possible_totals[0], self.table.dealer.partial_total), self.epsilon_greedy)
            if action == Action.HIT:  
                current_episode.append({
                    "state": State(player.chosen_total, self.table.dealer.partial_total),
                    "action": action,
                    "reward": 0
                })
                player.hand.add_card(next_card)
            else:   
                current_episode.append({
                    "state": State(player.chosen_total, self.table.dealer.partial_total),
                    "action": action,
                    "reward": 0
                })
                player.stand()
                
        return self.play_hand_possible(players, current_episode)
        
        

            
    
        

In [6]:
agent = Agent(
    learning_rate=0.1,
    discount_factor=0.99,
    epsilon_greedy=0.9,
    epochs=1000
)

In [7]:
player = blackjack.Player()
agent.table = blackjack.Table()
agent.table.add_player(player)
agent.table.start_turn()
agent.table.dealer.play(agent.table.deck)

episodes = agent.play_hand_possible(model_player=player, current_episode=[])
episodes

[{'state': State(hand_total=19, delear_partial_total=3),
  'action': <Action.STAND: 1>,
  'reward': -1}]